In [1]:
import numpy as np 
import string
import nltk
nltk.download('punkt')
nltk.download("stopwords")
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /home/darkness/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/darkness/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/darkness/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/darkness/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Pre Processing

### Text Retrival

In [2]:
f = open("Text.txt")
data = f.readlines()[6:]
f.close()

### Tokenization

In [3]:
# Sentence Splitting
sentences = []
for i in data:
    sentences += nltk.sent_tokenize(i)
# Word Splitting
words_in_sentence = []
for i in sentences:

    words_in_sentence.append(nltk.word_tokenize(i))

In [4]:
apostrophy = words_in_sentence[0][3] + '”“‘—'

In [5]:
words_in_sentence = [[i for i in j if i not in ( string.punctuation + apostrophy ) ] for j in words_in_sentence]
words_in_sentence = [[ i for i in j] for j in words_in_sentence]

In [6]:
Stop_word = stopwords.words('english') + list(string.ascii_lowercase) + list(string.ascii_uppercase)

words_nostop_in_sentence = []
for i in words_in_sentence:
    temp = [word for word in i if word not in Stop_word]
    words_nostop_in_sentence.append(temp)

### POS Tagging

In [7]:
POS = []
for i in words_nostop_in_sentence:
    POS.append(nltk.pos_tag(i))

### Lemmatization

In [8]:
root_per_sentence = []
obj = nltk.WordNetLemmatizer()

for i in words_nostop_in_sentence:
    temp  = [obj.lemmatize(j) for j in i ] 
    root_per_sentence.append(temp)

## TF -IDF VECTOR REPRESENTATION

In [9]:
unique_words = []
for i in root_per_sentence:
    for j in i :
        if j not in unique_words:
            unique_words.append(j)
unique_words = sorted(unique_words)

In [20]:
tf = []
idf = [0 for _ in range(len(unique_words))]
for i in range(len(root_per_sentence)):
    temp = [0 for _ in range(len(unique_words))]
    for j in root_per_sentence[i]:
        temp[unique_words.index(j)] = 1 / len(root_per_sentence[i])
        idf[unique_words.index(j)] += 1
    tf.append(temp) 

tf = np.array(tf)
idf = np.array(idf)
idf = np.array(np.log2(len(root_per_sentence))) - np.log2(idf)
idf = idf.reshape(1,-1)

In [11]:
# TF - IDF Vectorization
import pandas as pd
Data_TF_IDF = pd.DataFrame(index=["sentence-" + str(i+1) for i in range(len(root_per_sentence) )],columns=sorted(unique_words),data=tf)

In [12]:
vector_sentence1 = (np.array(Data_TF_IDF.loc['sentence-1']).reshape(1,-1) * idf)
vector_sentence2 = (np.array(Data_TF_IDF.loc['sentence-2']).reshape(1,-1) * idf)

In [13]:
def mag(vct):
    return np.sum(vct**2)

In [27]:
Sim_tf_idf = np.dot(vector_sentence1,vector_sentence2.T) / (mag(vector_sentence1) * mag(vector_sentence2))

### Similarity score(sent1,sent2) are 0.010788134718958942

## Bag of Words

In [21]:
tf_bow = []
for i in range(len(root_per_sentence)):
    temp = [0 for _ in range(len(unique_words))]
    for j in root_per_sentence[i]:
        temp[unique_words.index(j)] = 1 
    tf_bow.append(temp) 
tf_bow = np.array(tf_bow)

In [22]:
Data_bow = pd.DataFrame(index=["sentence-" + str(i+1) for i in range(len(root_per_sentence) )],columns=sorted(unique_words),data=tf_bow)

In [28]:
vector_sentence1_bow = (np.array(Data_bow.loc['sentence-1']).reshape(1,-1) )
vector_sentence2_bow = (np.array(Data_bow.loc['sentence-2']).reshape(1,-1) )
Sim_bow =  np.dot(vector_sentence1_bow,vector_sentence2_bow.T) / (mag(vector_sentence1_bow) * mag(vector_sentence2_bow))

### Similarity score(sent1,sent2) are 0.005208333333333333

### 

## END